In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
EPOCHS = 100

2025-10-04 17:14:09.466992: E itex/core/kernels/xpu_kernel.cc:38] XPU-GPU kernel not supported.


In [2]:
# Przygotowanie danych
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

In [3]:
#ile czasu zajmuje jeden krok dla poszczególnych wartości batch size?
batch_sizes = [8, 16, 32, 64, 128, 256]
histories = []

for batch_size in batch_sizes:
    print(f"Ustawiona wielość zbioru: {batch_size}")
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=5, batch_size=batch_size, validation_data=(test_images, test_labels))
    histories.append(history)

Ustawiona wielość zbioru: 8
Epoch 1/5
7500/7500 [==============================] - 22s 3ms/step - loss: 0.2117 - accuracy: 0.9377 - val_loss: 0.1093 - val_accuracy: 0.9675
Epoch 2/5
7500/7500 [==============================] - 20s 3ms/step - loss: 0.0949 - accuracy: 0.9709 - val_loss: 0.1036 - val_accuracy: 0.9693
Epoch 3/5
7500/7500 [==============================] - 25s 3ms/step - loss: 0.0646 - accuracy: 0.9797 - val_loss: 0.0912 - val_accuracy: 0.9725
Epoch 4/5
7500/7500 [==============================] - 25s 3ms/step - loss: 0.0494 - accuracy: 0.9837 - val_loss: 0.0920 - val_accuracy: 0.9750
Epoch 5/5
7500/7500 [==============================] - 25s 3ms/step - loss: 0.0409 - accuracy: 0.9869 - val_loss: 0.1037 - val_accuracy: 0.9728
Ustawiona wielość zbioru: 16
Epoch 1/5
3750/3750 [==============================] - 14s 4ms/step - loss: 0.2270 - accuracy: 0.9338 - val_loss: 0.1211 - val_accuracy: 0.9635
Epoch 2/5
3750/3750 [==============================] - 13s 3ms/step - loss: 0.0

In [4]:
rows = []
for bsz, hist in zip(batch_sizes, histories):
    df = pd.DataFrame(hist.history)             
    df['epoch'] = np.arange(1, len(df) + 1)      
    df['batch_size'] = bsz
    # uporządkuj kolumny:
    df = df[['batch_size', 'epoch'] + [c for c in df.columns if c not in ['batch_size', 'epoch']]]
    rows.append(df)

full = pd.concat(rows, ignore_index=True)
full.to_csv('mnist_histories.csv', index=False)
print('Zapisano do mnist_histories.csv')

Zapisano do mnist_histories.csv
